# Подержаные машины на eBay
---

### Описание

В этом проекте мы будем работать с набором данных о подержаных машинах из специалнього раздела немецкого eBay. Его можно скачать [тут](https://data.world/data-society/used-cars-data).

Ниже приведена расшировка данных.

* dateCrawled - дата занесения в базу.
* name - название машины.
* seller - продавец дилер или частное лицо.
* offerType - тип сделки
* price - цена.
* abtest - включен ли A/B тест.
* vehicleType - тип машины.
* yearOfRegistration - год первой регистрации авто.
* gearbox - тип трансмиссии.
* powerPS - мощность в лс.
* model - модель.
* kilometer - пробег.
* monthOfRegistration - месяц первой регистрации.
* fuelType - тип топлива.
* brand - марка.
* notRepairedDamage - есть ли неустранённые повереждения.
* dateCreated - дата создание объявления.
* nrOfPictures - количество картинок в нём.
* postalCode - почтовой индекс местоположения машины.
* lastSeenOnline - последняя дата, еогда объявление было онлайн.

Начнём с того, что импортируем необходимые библиотеки и прочитаем файл с данными


In [1]:
import numpy as np
import pandas as pd

autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [2]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [3]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Как можно заметить в названиях столбцов использован CamelCase. Для дальнейшей работы нам предпочтительно использовать snake_case(такой стиль гораздо более читабелен), поэтому переименуем интересующие нас столбцы.

In [4]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [25]:
autos.rename({'yearOfRegistration': 'registration_year', 'monthOfRegistration': 'registration_month',
              'notRepairedDamage': 'unrepaired_damage', 'dateCreated': 'ad_created','lastSeen': 'last_seen',
             'dateCrawled': 'date_crawled', 'vehicleType': 'vehicle_type'},
            axis=1, inplace=True)
autos.columns

Index(['date_crawled', 'name', 'price', 'abtest', 'vehicle_type',
       'registration_year', 'gearbox', 'powerPS', 'model', 'odometer_km',
       'registration_month', 'fuelType', 'brand', 'unrepaired_damage',
       'ad_created', 'postalCode', 'last_seen'],
      dtype='object')

## Очистка данных

Давайте внимательнее посмотрим на стоблцы, возможно некоторые их них абсолютно не информативны и их можно удалить

In [26]:
autos.describe(include='all')

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer_km,registration_month,fuelType,brand,unrepaired_damage,ad_created,postalCode,last_seen
count,38629,38629,38629.000000,38629,35873,38629.000000,37195,38629.000000,37011,38629.000000,38629.000000,36271,38629,32857,38629,38629.000000,38629
unique,37561,29747,NaN,2,8,NaN,2,NaN,242,NaN,NaN,7,40,2,74,NaN,31223
top,2016-03-11 22:38:16,BMW_318i,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,2016-04-07 03:16:17
freq,3,67,NaN,19944,10545,NaN,27820,NaN,3077,NaN,NaN,21911,8222,30472,1524,NaN,7
mean,NaN,NaN,7332.474359,NaN,NaN,2005.678713,NaN,128.912941,NaN,122780.035724,5.998757,NaN,NaN,NaN,NaN,51690.997256,NaN
std,NaN,NaN,13060.890754,NaN,NaN,86.681928,NaN,215.883262,NaN,40795.760641,3.591162,NaN,NaN,NaN,NaN,25689.793419,NaN
min,NaN,NaN,1000.000000,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,1067.000000,NaN
25%,NaN,NaN,2200.000000,NaN,NaN,2001.000000,NaN,80.000000,NaN,100000.000000,3.000000,NaN,NaN,NaN,NaN,31224.000000,NaN
50%,NaN,NaN,4350.000000,NaN,NaN,2005.000000,NaN,116.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,50858.000000,NaN
75%,NaN,NaN,8950.000000,NaN,NaN,2009.000000,NaN,160.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,72534.000000,NaN


К примеру столбцы `seller` и `offerType` имеют всего по 2 уникальных значения, при этом одно из них встречается в 49999 случаях из 50000. Из невозможно почерпнуть полезную инфу, соответственно их можно удалять.

Также можно заметить, что в столбцах `price` и `odometer` данные сохранены как текст, хотя для пробега и цены нужны числовые значения. Будем их переводить.

In [7]:
autos['price'].value_counts()

$0          1421
$500         781
$1,500       734
$2,500       643
$1,200       639
$1,000       639
$600         531
$3,500       498
$800         498
$2,000       460
$999         434
$750         433
$900         420
$650         419
$850         410
$700         395
$4,500       394
$300         384
$2,200       382
$950         379
$1,100       376
$1,300       371
$3,000       365
$550         356
$1,800       355
$5,500       340
$1,250       335
$350         335
$1,600       327
$1,999       322
            ... 
$73,900        1
$1,039         1
$13,888        1
$10,480        1
$13,680        1
$51,500        1
$2,549         1
$37,400        1
$2,175         1
$16,845        1
$4,180         1
$10,555        1
$88,900        1
$24,290        1
$37,950        1
$16,650        1
$30,650        1
$4,320         1
$15,186        1
$19,777        1
$30,987        1
$55,900        1
$1,820         1
$4,201         1
$175,000       1
$1,985         1
$925           1
$1,698        

In [8]:
autos['odometer'].value_counts()

150,000km    32424
125,000km     5170
100,000km     2169
90,000km      1757
80,000km      1436
70,000km      1230
60,000km      1164
50,000km      1027
5,000km        967
40,000km       819
30,000km       789
20,000km       784
10,000km       264
Name: odometer, dtype: int64

In [9]:
autos['nrOfPictures'].value_counts()

0    50000
Name: nrOfPictures, dtype: int64

При ближайнем рассмотрении становится понятно, что во всех ячейках столбца `nrOfPictures` записано одно и то же значение - **0**. Очередной бесполезный столбец.

In [37]:
autos['registration_year'].value_counts().sort_index()

1000       1
1001       1
1927       1
1929       1
1931       1
1934       2
1937       4
1938       1
1939       1
1941       2
1943       1
1948       1
1950       1
1951       2
1952       1
1953       1
1954       2
1955       2
1956       4
1957       2
1958       3
1959       6
1960      17
1961       6
1962       4
1963       8
1964       9
1965      17
1966      21
1967      25
        ... 
1999    1724
2000    2001
2001    2065
2002    2135
2003    2477
2004    2608
2005    2785
2006    2651
2007    2259
2008    2204
2009    2077
2010    1584
2011    1617
2012    1306
2013     795
2014     649
2015     361
2016     651
2017    1008
2018     398
2019       1
2800       1
4100       1
4500       1
5000       2
5911       1
6200       1
8888       1
9000       1
9999       2
Name: registration_year, Length: 91, dtype: int64

В столбце `registration_year` часть значений либо превышают текущий год, либо слишком маленькие, чтобы быть реальными.

Начнём чистить наш набор данных!

In [11]:
# Комментим после первого запуска
autos = autos.drop(['nrOfPictures', 'seller', 'offerType'], axis=1)

autos['price'] = autos['price'].str.replace(',', '').str.replace('$', '').astype('int64')

autos['odometer'] = autos['odometer'].str.replace(',', '').str.replace('km', '').astype('int64')

autos.rename(columns={'odometer': 'odometer_km'}, inplace=True)

Давайте внимательнее изучим данные по ценам и пробегу.

Сначала столбец `price`.

In [12]:
autos['price'].unique().shape

(2357,)

In [13]:
autos['price'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [32]:
autos['price'].value_counts().sort_index(ascending=False)

999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
194000      1
190000      1
180000      1
175000      1
169999      1
169000      1
163991      1
163500      1
155000      1
151990      1
145000      1
139997      1
137999      1
135000      1
130000      1
129000      1
128000      1
120000      2
         ... 
1212        2
1209        1
1201        2
1200      639
1199      126
1195        1
1190       37
1189        1
1180        4
1170        1
1169        1
1150      226
1149       10
1120        2
1119        1
1112        1
1111       39
1100      376
1099       44
1098        1
1095        3
1090        4
1080        6
1070        1
1059        1
1050       95
1049        6
1040        1
1039        1
1000      639
Name: price, Length: 2091, dtype: int64

Среди всех объявлений очень много записей с ценой в **0** долларов. Наверняка у продавцов были свои причины не указывать цену, но для нас такие записи бесполезны.

Также есть ряд записей с аномально большими и маленькими ценами, которые будут искажения в наш анализ.

Предположу, что подержанный автомобиль не должен стоить меньше **1000** и больше **1000000** долларов. Давайте уберём все выбивающиеся значения.

In [15]:
autos = autos[autos['price'].between(1000, 1000000)]

In [16]:
autos['price'].value_counts().sort_index(ascending=False).head()

999999    2
999990    1
350000    1
345000    1
299000    1
Name: price, dtype: int64

In [17]:
autos['price'].value_counts().sort_index(ascending=False).tail()

1050     95
1049      6
1040      1
1039      1
1000    639
Name: price, dtype: int64

Теперь посмотрим на столбец `odometer_km`.

In [18]:
autos['odometer_km'].unique().shape

(13,)

In [19]:
autos['odometer_km'].describe()

count     38629.000000
mean     122780.035724
std       40795.760641
min        5000.000000
25%      100000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [21]:
autos['odometer_km'].value_counts(ascending=False)

150000    23316
125000     4341
100000     1860
90000      1569
80000      1334
70000      1154
60000      1099
50000       986
40000       795
30000       748
20000       692
5000        507
10000       228
Name: odometer_km, dtype: int64

В столбце с пробегом на мой взгляд нет каких-либо странных значений.

## Работа с датами

В нашей таблице есть пять столбцов, которые так или иначе указывают на какие-либо даты.

Два столбца, указывающие на год и месяц регистрации, распознаются как `int64`. Нам это подходит(помним про странные значения в столбце с годами).

Три других столбца(`date_crawled`, `last_seen`, и `ad_created`) распознаются как `string`, поэтому нам необходимо привести их к более подходящему формату.

Для начала давайте посмотрим на то, как записаны даты в этих столбцах.

In [27]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


Во всех трёх случаях даты записаны однотипно, при этом непосредственно день записан в первых 10 символах.

Давайте используем это и посмотрим распределение объявлений по датам.

In [34]:
autos['date_crawled'].str[:10].value_counts(normalize=True ,dropna=False).sort_index()

2016-03-05    0.025551
2016-03-06    0.013876
2016-03-07    0.035129
2016-03-08    0.032618
2016-03-09    0.032463
2016-03-10    0.033317
2016-03-11    0.032799
2016-03-12    0.037381
2016-03-13    0.015998
2016-03-14    0.036631
2016-03-15    0.033628
2016-03-16    0.029071
2016-03-17    0.030495
2016-03-18    0.012840
2016-03-19    0.035129
2016-03-20    0.038158
2016-03-21    0.037304
2016-03-22    0.032514
2016-03-23    0.032204
2016-03-24    0.029020
2016-03-25    0.030521
2016-03-26    0.033110
2016-03-27    0.031401
2016-03-28    0.035362
2016-03-29    0.033990
2016-03-30    0.033058
2016-03-31    0.031401
2016-04-01    0.034611
2016-04-02    0.036294
2016-04-03    0.039142
2016-04-04    0.036863
2016-04-05    0.013358
2016-04-06    0.003262
2016-04-07    0.001501
Name: date_crawled, dtype: float64

In [35]:
autos['ad_created'].str[:10].value_counts(normalize=True ,dropna=False).sort_index()

2015-06-11    0.000026
2015-08-10    0.000026
2015-09-09    0.000026
2015-11-10    0.000026
2015-12-30    0.000026
2016-01-03    0.000026
2016-01-07    0.000026
2016-01-10    0.000052
2016-01-13    0.000026
2016-01-14    0.000026
2016-01-16    0.000026
2016-01-22    0.000026
2016-01-27    0.000078
2016-01-29    0.000026
2016-02-01    0.000026
2016-02-02    0.000052
2016-02-05    0.000052
2016-02-07    0.000026
2016-02-09    0.000026
2016-02-11    0.000026
2016-02-12    0.000052
2016-02-14    0.000052
2016-02-16    0.000026
2016-02-17    0.000026
2016-02-18    0.000052
2016-02-19    0.000078
2016-02-20    0.000026
2016-02-21    0.000052
2016-02-22    0.000026
2016-02-23    0.000104
                ...   
2016-03-09    0.032644
2016-03-10    0.032980
2016-03-11    0.033058
2016-03-12    0.037122
2016-03-13    0.017655
2016-03-14    0.034974
2016-03-15    0.033446
2016-03-16    0.029641
2016-03-17    0.030185
2016-03-18    0.013280
2016-03-19    0.034068
2016-03-20    0.038261
2016-03-21 

In [36]:
autos['last_seen'].str[:10].value_counts(normalize=True ,dropna=False).sort_index()

2016-03-05    0.001087
2016-03-06    0.003572
2016-03-07    0.004556
2016-03-08    0.006239
2016-03-09    0.008905
2016-03-10    0.009811
2016-03-11    0.011727
2016-03-12    0.022185
2016-03-13    0.008387
2016-03-14    0.011986
2016-03-15    0.014989
2016-03-16    0.015455
2016-03-17    0.026379
2016-03-18    0.007378
2016-03-19    0.014600
2016-03-20    0.019804
2016-03-21    0.019674
2016-03-22    0.020787
2016-03-23    0.017914
2016-03-24    0.018535
2016-03-25    0.017759
2016-03-26    0.016076
2016-03-27    0.014083
2016-03-28    0.019441
2016-03-29    0.020787
2016-03-30    0.023454
2016-03-31    0.022729
2016-04-01    0.023195
2016-04-02    0.024904
2016-04-03    0.024438
2016-04-04    0.023376
2016-04-05    0.131119
2016-04-06    0.234694
2016-04-07    0.139973
Name: last_seen, dtype: float64

Настало время вернуться к столбцу `registration_year` и его странным значениям.

In [38]:
autos['registration_year'].value_counts().sort_index()

1000       1
1001       1
1927       1
1929       1
1931       1
1934       2
1937       4
1938       1
1939       1
1941       2
1943       1
1948       1
1950       1
1951       2
1952       1
1953       1
1954       2
1955       2
1956       4
1957       2
1958       3
1959       6
1960      17
1961       6
1962       4
1963       8
1964       9
1965      17
1966      21
1967      25
        ... 
1999    1724
2000    2001
2001    2065
2002    2135
2003    2477
2004    2608
2005    2785
2006    2651
2007    2259
2008    2204
2009    2077
2010    1584
2011    1617
2012    1306
2013     795
2014     649
2015     361
2016     651
2017    1008
2018     398
2019       1
2800       1
4100       1
4500       1
5000       2
5911       1
6200       1
8888       1
9000       1
9999       2
Name: registration_year, Length: 91, dtype: int64

Очевидно, что значения **1000** и **1001** некорректные, никаких автомобилей в эти годы быть не могло. Также можно проверять каждый год из начала 20 века на достоверность, но давайте предположим, что они верны.

Также год регистрации не может быть больше года создания объявления: автомобиль должен был использоваться кем-то до этого момента. Самая поздняя дата создания - **07.04.2016**. Соответственно все значения года регистрации свыше **2016** не корректны.

Давайте удалим записи с некорректными годами.

In [43]:
autos = autos[autos['registration_year'].between(1900, 2016)]

autos['registration_year'].value_counts(normalize=True).sort_index()

1927    0.000027
1929    0.000027
1931    0.000027
1934    0.000054
1937    0.000108
1938    0.000027
1939    0.000027
1941    0.000054
1943    0.000027
1948    0.000027
1950    0.000027
1951    0.000054
1952    0.000027
1953    0.000027
1954    0.000054
1955    0.000054
1956    0.000108
1957    0.000054
1958    0.000081
1959    0.000161
1960    0.000457
1961    0.000161
1962    0.000108
1963    0.000215
1964    0.000242
1965    0.000457
1966    0.000564
1967    0.000672
1968    0.000699
1969    0.000484
          ...   
1987    0.001371
1988    0.002849
1989    0.003306
1990    0.005079
1991    0.005563
1992    0.005859
1993    0.005859
1994    0.007283
1995    0.011906
1996    0.014566
1997    0.021151
1998    0.034481
1999    0.046333
2000    0.053777
2001    0.055497
2002    0.057379
2003    0.066570
2004    0.070091
2005    0.074847
2006    0.071246
2007    0.060711
2008    0.059233
2009    0.055820
2010    0.042570
2011    0.043457
2012    0.035099
2013    0.021366
2014    0.0174

Из результата выше видно, что большинство автомобилей(~60%) из таблицы были зарегестрированы в 2000х.

## Исследуем марки автомобилей

Для начала давайте посмотрим какие вообще марки представлены в таблице и в каком количестве.

In [45]:
autos['brand'].value_counts(normalize=True)

volkswagen        0.210836
bmw               0.125346
mercedes_benz     0.111586
audi              0.097584
opel              0.089064
ford              0.058722
renault           0.037276
peugeot           0.027896
fiat              0.021070
skoda             0.019055
seat              0.017281
smart             0.016609
toyota            0.014620
mazda             0.014244
citroen           0.013894
nissan            0.013626
mini              0.010884
hyundai           0.010750
sonstige_autos    0.010454
volvo             0.008976
kia               0.007686
porsche           0.007471
honda             0.007337
mitsubishi        0.006880
chevrolet         0.006611
alfa_romeo        0.006235
suzuki            0.005724
dacia             0.003279
chrysler          0.003171
jeep              0.002768
land_rover        0.002634
jaguar            0.001854
subaru            0.001720
daihatsu          0.001693
saab              0.001371
daewoo            0.000914
trabant           0.000860
r

In [48]:
autos['brand'].value_counts(normalize=True)[:10].sum()

0.7984358622913812

Неудивительно, что в таблице весь топ занят немецкими марками, всё-таки мы рассматриваем данные из немецкой части ebay.

Далее будем работать только с первой десяткой: она закрывает практически 80% объявлений.

Посмотрим, какая средняя цена у машин каждого бренда.

In [80]:
brand_mean_price = {}

top_ten = autos['brand'].value_counts(normalize=True)[:10].index

for brand in top_ten:
    brand_mean_price[brand] = int(autos[autos['brand'] == brand]['price'].mean())
    
for brand in brand_mean_price:
    print(brand, '-', brand_mean_price[brand]) 

opel - 4219
renault - 3590
skoda - 6836
peugeot - 3955
fiat - 4008
volkswagen - 6898
bmw - 9119
audi - 10322
mercedes_benz - 9302
ford - 5786


Из результата видно, и это довольно предсказуемо, что наибольшую цену имеют машины марок: **BMW, Audi и Mersedes Benz**.

Самые дешёвые: **Opel**, **Renault**, **Fiat** и **Peugeot**.

Давайте посмотрим, есть ли связь между ценой и пробегом. Для начала получим средний пробег для каждой марки.

In [81]:
brand_mean_km = {}

for brand in top_ten:
    brand_mean_km[brand] = int(autos[autos['brand'] == brand]['odometer_km'].mean())
    
for brand in brand_mean_km:
    print(brand, '-', brand_mean_km[brand])

opel - 123952
renault - 121423
skoda - 110063
peugeot - 122341
fiat - 107901
volkswagen - 125771
bmw - 132001
audi - 127491
mercedes_benz - 130062
ford - 119622


А теперь давайте сравним цену и пробег.

In [82]:
bmp_series = pd.Series(brand_mean_price)
bmkm_series = pd.Series(brand_mean_km)

new_df = pd.DataFrame(bmp_series, columns=['mean_price'])
new_df['mean_km'] = bmkm_series
new_df

,mean_price,mean_km
audi,10322,127491
bmw,9119,132001
fiat,4008,107901
ford,5786,119622
mercedes_benz,9302,130062
opel,4219,123952
peugeot,3955,122341
renault,3590,121423
skoda,6836,110063
volkswagen,6898,125771


Средний пробег у автомобилей всех марок примерно одинаков, разницы в два раза как с ценами нет. Следовательно можно сделать вывод, что разница в цене вызвана только маркой самого автомобиля.